In [1]:
# Imports
import os
import sys
import cmath
import math
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import pandas as pd
import ltspice
import sympy as sp
from sympy.utilities.lambdify import lambdify
from scipy import signal
%matplotlib inline
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from matplotlib.ticker import LogLocator

https://training.ti.com/how-design-transimpedance-amplifier-circuits

https://www.tij.co.jp/jp/lit/an/snoa942a/snoa942a.pdf?ts=1622056332116&ref_url=https%253A%252F%252Fwww.google.com%252F

https://www2.eecs.berkeley.edu/Pubs/TechRpts/2020/EECS-2020-146.pdf

<hr>

**IEEE**

https://people.engr.tamu.edu/spalermo/ecen689_oi/2004_rgc_tia_park_jssc.pdf

<hr>

Review of Modern CMOS Transistor

https://www.mdpi.com/2079-9292/8/10/1073/pdf

**Hobbs and AoE**

https://electrooptical.net/static/oldsite/talks/GettingPDRight11.pdf

https://x.artofelectronics.net/wp-content/uploads/2019/11/4xp3_TIA.pdf

<hr>

**Cascode frequency response**

- The photodiode signal current also sees an input impedance of $1/g_{m}$ (at low frequencies) due to the cascode device
- At high frequencies, the diode capacitance $C_d$ shunts the signal current, reducing the current going into the TIA
- The transfer function seen by the photodiode current is
$$
\begin{equation}
i_{in} = \dfrac{1}{1 + sC_d/g_{m}}i_d 
\end{equation}
$$

**Biased cascode noise**

- The bias current of the cascode stage is developed based on the value of $R_E$ and the supply voltage, and is approximately given by $I_{C} = (V_{EE} - 0.6V)/R_E$
 
- For a bias current of, say, $200\mu A$ and a supply voltage of $\pm 15V$, this gives $R_E \approx 72k\Omega$ ($R_C$ will be a similar value)
 
- Still trying to limit the PSD of the thermal noise to $10\%$ of the shot noise limit sets a lower bound on the signal current:
 
\begin{equation}
i_d \geq 10\cdot\dfrac{4kT}{2q\cdot(R_E + R_C)} \approx 3.6\mu A
\end{equation}

 - The output noise contributed by $Q_1$ is minimized by $R_E$, and given by
 
\begin{equation}
i_{nc1,out}^2 = \dfrac{2qI_C}{(1+g_{m1}R_E)^2}
\end{equation}

**Regulated cascode**

- We can achieve both low noise and low input impedance by applying feedback and decreasing $Q_1$'s collector current to reduce both its shot noise and the thermal noise from $R_E$ and $R_{C1}$
 
- The regulated (or, gain-boosted) cascode works by amplifying $Q_1$'s emitter voltage and feeding it back to its base, effectively multiplying the transconductance of $Q_1$ by the voltage gain of the common-emitter amplifer formed by $Q_2$, $R_{C2}$
 
- The input impedance of the regulated cascode is given by
 
\begin{equation}
R_{in} = \dfrac{1}{g_{m1}(1+g_{m2}R_{C2})}
\end{equation}

- The result is an increase in the input pole frequency by the factor $g_{m2}R_{C2}$ *without* a substantial increase in $Q_1$'s collector current (or the resulting base current shot noise)
 
- $Q_1$'s shot noise is reduced even further by the $g_m$-boosting action of the $Q_2$ common-emitter stage, and is given approximately by

\begin{equation}
i_{nc1,out}^2 \approx \dfrac{2qI_{C1}}{(1+g_{m2}R_{C2}g_{m1}R_E)^2}
\end{equation}
 
- $Q_2$'s shot noise sees a gain of
 
\begin{align}
i_{nc2,out} &= \dfrac{g_{m1}R_{C2}}{1+g_{m1}R_E}i_{nc2}\\
\end{align}

<hr>

| Parameter                                             | Specification |
|:------------------------------------------------------|:--------------|
| Transimpedance gain (full signal path)                | $1$ $V/\mu A$ |
| Transimpedance bandwidth ($f_{3dB}$)                  | $1$ $MHz$     |
| Minimum photodiode current ($i_d$)                    | $2$ $\mu A$   |
| Photodiode capacitance ($C_d$)                        | $150$ $pF$    |
| Closed-loop gain magnitude peaking                    | $0$  $dB$     |
| $rms$ noise figure ($F = 1 + i_n^2/i_{ns}^2$)         | $\leq 0.5$ $dB$ |
| Power dissipation ($I_{DD} \cdot (V_{CC} + V_{EE}) $) | Optimize $mW$ |
| Cost-per-unit (1000 units)                            | Optimize $\$$ |

<hr>

**TIA by itself** 

$C_{in}$ is fixed to photodiode (150 pF) + TIA (3-20 pF)

$R_f$ is determined by gain (1e6)

$C_f \approx \sqrt{\dfrac{2\cdot C_{in}}{R_f\omega_t}}$

Since $R_f$ and is $C_{in}$ fixed, $C_f$ is a function of $\omega_t$

$\omega_{3dB} = \sqrt{\frac{\omega_t}{R_f  C_{in}}} = \sqrt{65K \cdot \omega_t}$

And $\omega_{3dB}$ is a function of $\omega_t$. 
<hr>

In [11]:
k = 1.38e-23
T = 300
q = 1.602e-19
V_T = k*T/q

# Photodiode
Cd = 150*1e-12
Id = 2*1e-6

# TIA LF356
A0 = 200*1e3
C_TIA = 3*1e-12
ft = 4*1e6
# Vp = 7
# Vn = -4

# Parameters
Cin = Cd + C_TIA
wt = 2*np.pi*ft
Rf = 100*1e3

The closed-loop transfer function of the TIA is second order, and can be expressed as

$\begin{eqnarray}
&& H(s)  &=& {\frac {\omega _{0}^{2}}{s^{2}+2\zeta \omega _{0}\cdot s+\omega _{0}^{2}}} \\[0.5em]
&&       &=& {\frac {\omega _{0}^{2}}{s^{2}+\frac{s\cdot\omega_0}{Q}+\omega _{0}^{2}}} \\[0.5em]
&&       &=& {\frac {1}{\frac{s^{2}}{\omega_0^2}+\frac{s\cdot\omega_0}{Q}+1}}  \\[0.5em]
&& \frac{V_{out}}{i_{in}} &\approx& -R_f \frac{1}{1 + s R_f C_f + \frac{s^2}{\omega_u}R_f  C_{in}} \\[0.5em]
&& \frac{1}{\omega_0^2} &=& \frac{R_f  C_{in}}{\omega_u} \\[0.5em]
&& \omega_0^2 &=& \frac{\omega_u}{R_f  C_{in}} \\[0.5em]
&& \omega_0 &=& \sqrt{\frac{\omega_u}{R_f  C_{in}}} \\[0.5em]
&& \omega_u &\approx& \omega_t
\end{eqnarray}$

<hr>

$\begin{eqnarray}
& \omega_{3dB_{CL}} & &=& \omega_0 \\[0.5em]
& \omega_{3dB_{CL}}& &=& \sqrt{\frac{\omega_u}{R_f  C_{in}}} \\[0.5em]
&& &=& \sqrt{\frac{\omega_t}{R_f  C_{in}}} \\[0.5em]
\end{eqnarray}$

<hr>

That is, in order to achieve a "well-behaved response" we must set $\omega_z = \sqrt{\omega_{p,cl}\omega_t}$

$$
\begin{eqnarray}
\omega_z &=& \frac{1}{R_fC_f} \\
\omega_t && && \text{ [transit frequency]} \\
\omega_{p,cl} &=& \frac{1}{R_f(C_f+C_{in})} && \text{ [parasitic pole]}\\
\end{eqnarray}
$$

"The transimpedance rolls off somewhat
earlier than this, since it depends on the
magnitudes of the impedances of the feed-
back elements, and not merely on their ratio.
Without getting into lots of algebra, we
lose a factor of between √2 and 2 in band-
width, depending on the details of the 
frequency compensation scheme"

$\begin{eqnarray}
&& f_{-3dB} &=& \frac{\sqrt{f_{RC}f_T}}{2}
\end{eqnarray}$

<hr>

In [26]:
f_RC = 1/(Rf*Cin)
f3db = np.sqrt(f_RC * ft)/2
print(f'Worst case f3db = {round(f3db)} Hz')

Worst case f3db = 255655 Hz


<hr>

For a damping factor of $\zeta = 1/\sqrt{2}$ (or equivalently, $Q=1/\sqrt{2}$, a maximally flat response), and assuming $C_{in}>>C_f$, the required value of $C_f$ is approximately
$$ 
\begin{equation}
C_f \approx \sqrt{\dfrac{2\cdot C_{in}}{R_f\omega_t}} = \dfrac{1}{R_f}\sqrt{\dfrac{2\cdot R_f C_{in}}{\omega_t}}
\end{equation}
$$

In [18]:
Cf = np.sqrt(2*Cin/(Rf*wt))
print(f'Cf = {round(Cf*1e12,3)/(1e12)}')

Cf = 1.1034e-11


<hr>

Assuming an ideal (infinite-bandwidth) opamp, the noise transfer function is the input/output relationship for the amplifier voltage noise
$$
\begin{eqnarray}
\dfrac{e_{na,out}}{e_{na}} &=& \left(1+\dfrac{Z_f}{Z_{in}}\right)\\ 
\\
&=& \dfrac{1+s(C_{in}+C_f)R_f}{1+sC_fR_f}
\end{eqnarray}
$$


 - This expression has a zero at $1/R_f(C_{in}+C_f) \approx 1/R_fC_{in}$, which results in "peaking" of the output noise due to $e_{na}$ and significantly degrades the noise performance of the TIA
 
 - The noise due to $e_{na}$ rises with frequency, as the impedance of $C_{in}$ decreases

0 dB Closed-loop gain magnitude peaking results in 

$\begin{eqnarray}
&& 1+s(C_{in}+C_f)R_f &=& 1+sC_fR_f
\end{eqnarray}$

Such that $C_{in} \geq 100 C_f$

$C_f \leq 1.5$ pF 

<hr>

**Noise Figure**

$$\begin{eqnarray}
&& F = 1 + i_n^2/i_{ns}^2
\end{eqnarray}$$

The photodiode produces a shot noise current with power spectral density given by 
$$
\begin{equation}
i_{ns}^2 = 2qi_d
\end{equation}
$$

$\begin{eqnarray}
&& i_n^2 &=& \left[\frac{\text{op amp noise}}{G}\right]^2 + \left[\frac{\text{thermal noise}(R_f)}{G}\right]^2 \\[0.5em]
&& i_n^2 &=& \left[\frac{e_{na,out}}{G} \right]^2 + \left[ \frac{\sqrt{4 kT R_f}}{G}\right]^2 \\[0.5em]
&& G &=& R_f \\[0.5em]
&& i_n^2 &=& \left[\frac{e_{na,out}}{R_f} \right]^2 + \left[ \frac{\sqrt{4 kT R_f}}{R_f}\right]^2 \\[0.5em]
&& i_n^2 &=& \left[\frac{1}{R_f} \cdot \dfrac{1+s(C_{in}+C_f)R_f}{1+sC_fR_f} \right]^2 + \left[ \frac{\sqrt{4 kT R_f}}{R_f}\right]^2 \\[0.5em]
&& i_n^2 &\approx& \left[\dfrac{sC_{in}}{1+sC_fR_f} \right]^2 + \left[ \sqrt{\frac{4 kT}{R_f}} \right]^2 \\[0.5em]
\end{eqnarray}$

<hr>

In [ ]:
w = np.logspace(1, 8, num=700)
Cin = 100e-12
Rf = 100e3
wt = 10e6
Cf = np.sqrt(2*Cin/Rf/wt)
ena = 5e-9
G_noise = signal.TransferFunction([(Cin+Cf)*Rf, 1], [Cf*Rf, 1])
w, mag, phase = signal.bode(G_noise, w=w)
plot_logxy(w/2/np.pi, 1e9*10**(mag/20)*ena, 'Frequency [Hz]', r'Output Noise [$nV/\sqrt{Hz}$]')

$\begin{eqnarray}
&& A_{Vcl} &=& \frac{A_{Vol}}{1 + \frac{A_{Vol}}{1 + jv C_d Z_f}}
\end{eqnarray}$

<hr>